In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [19]:
df_fake=pd.read_csv('Fake.csv')
df_true=pd.read_csv('True.csv')

In [20]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [21]:
df_true.head(5)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [22]:
df_true.shape

(21417, 4)

In [23]:
df_fake.shape

(23481, 4)

In [24]:
df_true['label']=1
df_fake['label']=0

In [25]:
df_true.shape

(21417, 5)

In [27]:
fake_data_manual_testing=df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i],axis=0,inplace=True)


true_data_manual_testing=df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i],axis=0,inplace=True)

In [28]:
df_true.shape

(21407, 5)

In [29]:
df_fake.shape

(23471, 5)

In [30]:
fake_data_manual_testing['label']=0
true_data_manual_testing['label']=1

C:\Users\DELL\AppData\Local\Temp\ipykernel_8272\123336929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_data_manual_testing['label']=0
C:\Users\DELL\AppData\Local\Temp\ipykernel_8272\123336929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_data_manual_testing['label']=1


In [32]:
news_data=pd.concat([df_fake,df_true],axis=0)

In [34]:
news_data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [35]:
news_data.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

In [36]:
news_data.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [37]:
news_data.shape

(44878, 5)

In [38]:
ndf=news_data.drop(['title','subject','date'],axis=1)

In [39]:
ndf.shape

(44878, 2)

In [40]:
ndf.columns

Index(['text', 'label'], dtype='object')

In [41]:
ndf=ndf.sample(frac=1)
ndf.head()

,text,label
12790,"BRUSSELS (Reuters) - Martin Selmayr, a top aid...",1
19765,BERLIN (Reuters) - Germans will vote on Sunday...,1
3690,Sean Hannity wants to destroy freedom of the p...,0
16658,NAIROBI (Reuters) - Kenya s opposition leader ...,1
530,When Afro-Latina journalist Ilia Calder n show...,0


In [42]:
ndf.reset_index(inplace=True)
ndf.drop(['index'],axis=1,inplace=True)

In [43]:
ndf.head()

,text,label
0,"BRUSSELS (Reuters) - Martin Selmayr, a top aid...",1
1,BERLIN (Reuters) - Germans will vote on Sunday...,1
2,Sean Hannity wants to destroy freedom of the p...,0
3,NAIROBI (Reuters) - Kenya s opposition leader ...,1
4,When Afro-Latina journalist Ilia Calder n show...,0


In [44]:
def func(text):
    text=text.lower()
    text=re.sub('\[.?\*]','',text)
    text=re.sub("\\W"," ",text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    text=re.sub('https?://\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
    return text

In [45]:
ndf['text']=ndf['text'].apply(func)

In [46]:
X=ndf['text']
y=ndf['label']

In [47]:
sample_text = """
This is an example text with [some content] inside square brackets.
It includes a URL: https://example.com and some punctuation! Also, numbers like 1234.
"""
cleaned_text = func(sample_text)
print(cleaned_text)

 this is an example text with  some content  inside square brackets  it includes a url  https   example com and some punctuation  also  numbers like   


In [48]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
X_train_vec=vectorizer.fit_transform(X_train)
X_test_vec=vectorizer.transform(X_test)

In [56]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train_vec,y_train)

LogisticRegression()

In [57]:
pred=model.predict(X_test_vec)

In [58]:
accuracy=accuracy_score(y_test,pred)

In [59]:
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.99


In [60]:
model.score(X_test_vec,y_test)

0.9868092691622103

In [61]:
print(classification_report(y_test , pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5855
           1       0.99      0.99      0.99      5365

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [63]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(n_estimators=100,random_state=42)
rf_model.fit(X_train_vec,y_train)

RandomForestClassifier(random_state=42)

In [64]:
pred_rf=rf_model.predict(X_test_vec)

In [65]:
rf_model.score(X_test_vec,y_test)

0.9883244206773618

In [66]:
acc=accuracy_score(y_test,pred_rf)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.99


In [70]:
import pickle
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)